In [2]:
from datasets import load_dataset
import json
from dataclasses import dataclass
from enum import Enum
import zlib
import pickle
from tqdm import tqdm
from src.utils.utils import evaluate_generations, codegen_metrics
from src.tasks.debug.lcb_debug import CodeGenerationProblem

dataset = load_dataset("huypn16/LCB-R-F")["test"]

/datadrive5/huypn16/anaconda3/envs/vexec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset[0]

{'question_title': 'Fill the Gaps',
 'question_content': 'We have a sequence of length N consisting of positive integers: A=(A_1,\\ldots,A_N). Any two adjacent terms have different values.\nLet us insert some numbers into this sequence by the following procedure.\n\n- If every pair of adjacent terms in A has an absolute difference of 1, terminate the procedure.\n- Let A_i, A_{i+1} be the pair of adjacent terms nearest to the beginning of A whose absolute difference is not 1.\n- If A_i < A_{i+1}, insert A_i+1,A_i+2,\\ldots,A_{i+1}-1 between A_i and A_{i+1}.\n- If A_i > A_{i+1}, insert A_i-1,A_i-2,\\ldots,A_{i+1}+1 between A_i and A_{i+1}.\n\n\n- Return to step 1.\n\nPrint the sequence when the procedure ends.\n\nInput\n\nThe input is given from Standard Input in the following format:\nN\r\nA_1 A_2 \\ldots A_N\n\nOutput\n\nPrint the terms in the sequence when the procedure ends, separated by spaces.\n\nConstraints\n\n\n- 2 \\leq N \\leq 100\n- 1 \\leq A_i \\leq 100\n- A_i \\neq A_{i+1}\n

In [4]:
class PromptConstants:
    SYSTEM_MESSAGE_GENERIC = f"You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests. You should think step-by-step logically before returning final the program. The program should only include function definition with parameter list in order."

    SYSTEM_MESSAGE_GEMINI = f"You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests. You will NOT return anything except for the program. Do NOT use system calls like `exit` in the generated program."

    SYSTEM_MESSAGE_DEEPSEEK = f"You are an AI programming assistant, utilizing the DeepSeek Coder model, developed by DeepSeek Company, and you answer questions related to computer science."

    SYSTEM_MESSAGE_MAGIC = f"You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable responses to user instructions.\n\n@@ Instruction\n"

    SYSTEM_MESSAGE_WIZARD = "Below is an instruction that describes a task. Write a response that appropriately completes the request."

    SYSTEM_MESSAGE_PHIND = f"""You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests. You will NOT return anything except for the program. Put your fixed program within code delimiters, for example: 
```python 
# YOUR CODE HERE
```"""

    SYSTEM_MESSAGE_CODEQWEN = (
        f"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user"
    )

    FORMATTING_MESSAGE_WITH_STARTER_CODE = "You will use the following starter code to write the solution to the problem and enclose your code within delimiters."

    FORMATTING_WITHOUT_STARTER_CODE = "Read the inputs from stdin solve the problem and write the answer to stdout (do not directly test on the sample inputs). Enclose your code within delimiters as follows."
    

In [5]:
import anthropic
import random

def get_generic_question_template_answer(question: CodeGenerationProblem):
    prompt = f"### Question:\n{question.question_content}\n\n"
    if question.starter_code:
        prompt += (
            f"### Format: {PromptConstants.FORMATTING_MESSAGE_WITH_STARTER_CODE}\n"
        )
        prompt += f"```python\n{question.starter_code}\n```\n\n"
    else:
        prompt += f"### Format: {PromptConstants.FORMATTING_WITHOUT_STARTER_CODE}\n"
        prompt += "Your reasoning: ....```python\n# YOUR CODE HERE\n```\n\n"
    prompt += f"### Answer: (use the provided format with backticks)\n\n"
    return prompt

In [6]:
import re
pattern = r"```python(.*?)```"  
def run(problem: CodeGenerationProblem, client: anthropic.Anthropic, num_samples: int = 5):
    generations = []
    raw_responses = []
    chat_messages = [
        {
            "role": "user",
            "content": get_generic_question_template_answer(problem),
        },
    ]
    for i in range(num_samples):
        try:
            response = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                system=PromptConstants.SYSTEM_MESSAGE_GENERIC,
                messages=chat_messages,
                max_tokens=2048
            )
            solution = response.content[0].text
        except:
            solution = ""
        match = re.search(pattern, solution, re.DOTALL)
        if match:
            python_code = match.group(1).strip()
            generations.append(python_code)
            raw_responses.append(solution)
    return generations, raw_responses

In [7]:
def run_cause_runtime_error(problem, responses, client):
    generations = []
    for response in responses:
        if len(response) < 10:
            continue
        prompt = f"### Question:\n{problem.question_content}\n\n"
        prompt += f"### Input-Output: {problem.public_test_cases}\n\n"
        prompt += f"### Target code {response}"
        chat_messages = [
            {
                "role": "user",
                "content": response,
            }
        ]
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            system="You are an expert in python code mutation, you will be given a code snippet as well as an input and you need to modify it to cause a runtime error.",
            messages=chat_messages,
            max_tokens=2048
        )
        solution = response.content[0].text
        match = re.search(pattern, solution, re.DOTALL)
        if match:
            python_code = match.group(1).strip()
            generations.append(python_code)
    return generations

In [8]:
import os
from tqdm import tqdm 
generations_all = []
samples = []
responses_all = []
client = anthropic.Anthropic(api_key=)

for idx in tqdm(range(len(dataset))):
# for idx in range(1):
    problem = CodeGenerationProblem(dataset[idx])
    generations = []
    generations, raws = run(problem, client, num_samples=3)
    buggy_generations = run_cause_runtime_error(problem, generations, client)
    generations_all.append(buggy_generations)
    samples.append(problem.get_evaluation_sample())
    
results, metadata = evaluate_generations(samples, generations_all, num_process_evaluate=32, timeout=6)
metric = codegen_metrics(samples, generations_all)

  0%|          | 0/153 [00:00<?, ?it/s]

  7%|▋         | 11/153 [00:08<01:10,  2.02it/s]

Unreachable
Unreachable
Unreachable
Unreachable
Unreachable


 67%|██████▋   | 103/153 [00:17<00:03, 14.70it/s]

alarm went off


...

 17%|█▋        | 80/459 [00:28<02:06,  2.99it/s]

Unreachable
Unreachable
Unreachable
Unreachable
Unreachable


 85%|████████▍ | 388/459 [00:39<00:01, 43.81it/s]Fatal Python error: Segmentation fault

Current thread 0x00007ffb04f62180 (most recent call first):
  File "<string>", line 53 in <genexpr>
  File "<string>", line 53 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
  File "<string>", line 54 in count_powerful
 

alarm went off
alarm went off
alarm went off
alarm went off


100%|██████████| 459/459 [00:48<00:00,  9.55it/s]


In [ ]:
with open("")

In [22]:
generations_all[0][0]

'def find_pair_to_insert(seq):\n    for i in range(len(seq) - 1):\n        if abs(seq[i] - seq[i+1]) != 1:\n            return i\n    return -1\n\ndef insert_numbers(seq, index):\n    a, b = seq[index], seq[index+1]\n    if a < b:\n        to_insert = list(range(a + 1, b))\n    else:\n        to_insert = list(range(a - 1, b, -1))\n    return seq[:index+1] + to_insert + seq[index+1:]\n\n# Read input\nN = int(input())\nA = list(map(int, input().split()))\n\n# Main logic\nwhile True:\n    index = find_pair_to_insert(A)\n    if index == -1:\n        break\n    A = insert_numbers(A, index)\n    \n    # Introduce potential division by zero\n    A[0] = A[0] // (A[-1] - A[0])\n\n# Print result\nprint(*A)'

In [9]:
metric

[{'pass@1': 0.02832244008714597,
  'detail': {'pass@1': {0: 0.0,
    1: 0.0,
    2: 0.0,
    3: 0.0,
    4: 0.0,
    5: 0.0,
    6: 0.0,
    7: 0.0,
    8: 0.0,
    9: 0.0,
    10: 0.0,
    11: 0.0,
    12: 0.0,
    13: 0.0,
    14: 0.0,
    15: 0.0,
    16: 0.0,
    17: 0.0,
    18: 0.0,
    19: 0.0,
    20: 0.0,
    21: 0.0,
    22: 0.0,
    23: 0.0,
    24: 0.0,
    25: 0.0,
    26: 0.0,
    27: 0.0,
    28: 0.0,
    29: 0.0,
    30: 0.0,
    31: 0.0,
    32: 0.0,
    33: 0.0,
    34: 0.0,
    35: 0.0,
    36: 0.0,
    37: 0.0,
    38: 0.0,
    39: 0.0,
    40: 0.0,
    41: 0.0,
    42: 0.0,
    43: 0.0,
    44: 0.0,
    45: 0.0,
    46: 0.0,
    47: 0.0,
    48: 0.0,
    49: 0.0,
    50: 0.0,
    51: 0.0,
    52: 0.0,
    53: 0.0,
    54: 0.0,
    55: 0.0,
    56: 0.0,
    57: 1.0,
    58: 0.0,
    59: 0.0,
    60: 0.0,
    61: 0.0,
    62: 0.0,
    63: 0.0,
    64: 0.0,
    65: 0.0,
    66: 0.0,
    67: 0.0,
    68: 0.0,
    69: 0.0,
    70: 0.0,
    71: 0.0,
    72: 0.0,
    73: 

In [10]:
valid_ids = {}
error_messages = []
for id_instance, instance in enumerate(metric[2]):
    messages = []
    if id_instance not in valid_ids:
        valid_ids[id_instance] = []
    for id_sample, sample in enumerate(instance):
        if "Runtime Error" in sample:
            valid_ids[id_instance].append(id_sample)
            messages.append(sample)
        
    error_messages.append(messages)
    

In [ ]:
# valid_ids = {k: v for k, v in valid_ids.items() if len(v) > 0}
# error_messages = [error_messages[k] for k in valid_ids.keys()]

In [16]:
# # Initialize an empty list to store the result
result = []
total_samples = 0
# Iterate over each key `i` in `valid_ids`
for i in valid_ids.keys():
    # Check if `valid_ids[i]` has elements
    if len(valid_ids[i]) > 0:
        inner_list = []
        for valid_id in valid_ids[i]:
            inner_list.append(generations_all[i][valid_id])
                
        result.append(inner_list)
        total_samples += len(inner_list)
    else:
        result.append([])
print(total_samples)

312


In [17]:
dataset = dataset.add_column("failed_solution_time_error", result)

for idx in range(len(dataset)):
    dataset[idx]["error_messages_time_error"] = error_messages[idx]

In [ ]:
# for instance in dataset:
#     if len(instance["failed_solution"]) != len(instance["error_messages"]):
#         print(len(instance["failed_generations"]))
#         print(len(instance["error_messages"]))

In [ ]:
# error_messages[0]

In [2]:
import anthropic
import re
import os

pattern = r"```python(.*?)```"  
SYSTEM_MESSAGE_GENERIC = f"You are an expert Python programmer. You will be given a question (problem specification), a buggy program and its error message, you will generate a correct Python program that matches the specification, fix the original program and passes all the tests. You should think step-by-step logically before returning final the program. The program should only include function definition with parameter list in order."

def get_template_answer(question: CodeGenerationProblem, buggy_prorgam: str, error_message: str):
    prompt = f"### Question:\n{question.question_content}\n\n"
    prompt += f"### Buggy program:\n```python\n{buggy_prorgam}\n```\n\n"
    prompt += f"### Error message:\n{error_message}\n\n"
    prompt += "Your reasoning: ....```python\n# YOUR CODE HERE\n```\n\n"
    return prompt
    
def run(problem: CodeGenerationProblem, buggy_program: str, error_message: str):
    client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
    chat_messages = [
        {
            "role": "user",
            "content": get_template_answer(problem, buggy_program, error_message),
        },
    ]
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            system=SYSTEM_MESSAGE_GENERIC,
            messages=chat_messages,
            max_tokens=2048
        )
        solution = response.content[0].text
    except:
        solution = ""
        
    match = re.search(pattern, solution, re.DOTALL)
    if match:
        python_code = match.group(1).strip()
    else:
        python_code = ""
    return python_code

In [6]:
generations_all = []
samples = []
responses_all = []

for idx in tqdm(range(len(dataset))):
# for idx in range(1):
    problem = CodeGenerationProblem(dataset[idx])
    generations = []
    for j, buggy_program in enumerate(dataset[idx]["failed_solution"]):
        error_message = dataset[idx]["error_messages"][j]     
        generations.append(run(problem, buggy_program, error_message))
    generations_all.append(generations)
    samples.append(problem.get_evaluation_sample())
    
results, metadata = evaluate_generations(samples, generations_all, num_process_evaluate=32, timeout=6)
metric = codegen_metrics(samples, generations_all)

  0%|          | 0/153 [00:00<?, ?it/s]

In [5]:
from src.codetransform.next import execute_and_trace
from tqdm import tqdm

In [12]:
import anthropic
import re
import os

pattern = r"```python(.*?)```"  
SYSTEM_MESSAGE_GENERIC = f"You are an expert Python programmer. You will be given a question (problem specification), a buggy program with annotation of intermediate variables and its error message, you will generate a correct Python program that matches the specification, fix the original program based on control flow and error message to pass all the tests. You should think step-by-step logically before returning final the program. The program should only include function definition with parameter list in order."

def get_template_answer_next(question: CodeGenerationProblem, annotated_program: str, error_message: str):
    
    
    prompt = f"### Question:\n{question.question_content}\n\n"
    prompt += f"### Annotated Buggy program:\n```python\n{annotated_program}\n```\n\n"
    prompt += f"### Error message:\n{error_message}\n\n"
    prompt += "Your reasoning: ....```python\n# YOUR CODE HERE\n```\n\n"
    return prompt
    
def run_next(problem: CodeGenerationProblem, buggy_program: str, error_message: str):
    client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
    chat_messages = [
        {
            "role": "user",
            "content": get_template_answer_next(problem, buggy_program, error_message),
        },
    ]
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            system=SYSTEM_MESSAGE_GENERIC,
            messages=chat_messages,
            max_tokens=2048
        )
        solution = response.content[0].text
    except:
        solution = ""
        
    match = re.search(pattern, solution, re.DOTALL)
    if match:
        python_code = match.group(1).strip()
    else:
        python_code = ""
    return python_code

In [7]:
from together import Together

client = Together()

def run_model(prompt):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
    )
    return completion.choices[0].message.content

In [8]:
def parse(program):
    pattern = r"```python(.*?)```"
    match = re.search(pattern, program, re.DOTALL)
    if match:
        python_code = match.group(1).strip()
    else:
        python_code = ""
    return python_code
    
def transform_program(program, input_output):
    prompt = f"""Given following program and input-output examples, firstly put any defined variable into the solution method or function. Remove any stdin expression and inplace directly the input into method call or function in a single assert\n\n. Only return the python transformed program only. The transformed program should include solve python code and a single assert, when the input parameters are inside method or function call in this assert statement. For example,
original program:
MOD = 123
class Solution:
    def add(self, a, b):
        return a + b + MOD

input: 1, 2
expected: 125
transformed program:
class Solution:
    def add(self, a, b):
        return a + b + 123

assert Solution().add(1, 2) == 125

original program:
N = int(input())
def add(a):
    return a + 1

print(add(N))

input: 1
expected: 2
transformed program:
def add(a):
    return a + 1

assert add(1) == 2
original program:
{program}
output, input and expected output: {input_output}
transformed program:"""
    # testable_program = model.generate_content([prompt]).text
    testable_program = run_model(prompt)
    testable_program = parse(testable_program)
    annotated_program = execute_and_trace(testable_program)

    return annotated_program, testable_program

In [9]:
# import time
# max_retries = 2
# wait_time = 10
# next_programs = []
# testable_programs = []

# for idx in tqdm(range(len(dataset))):
#     next_annotated_programs_per_problem = []
#     testable_programs_per_problem = []
#     for j, buggy_program in enumerate(dataset[idx]["failed_solution"]):
#         error_message = dataset[idx]["error_messages"][j]
#         for attempt in range(max_retries):
#             try:
#                 # Try to transform the buggy program
#                 next_annotated_program, testable_program = transform_program(buggy_program, error_message)
#                 break  # Exit the loop if successful
#             except Exception as e:
#                 print(f"Attempt {attempt + 1} failed: {e}")
#                 next_annotated_program = ""
#                 testable_program = ""
#                 if attempt < max_retries - 1:  # Wait only if there are retries left
#                     print(f"Retrying in {wait_time} seconds...")
#                     time.sleep(wait_time)
#                 else:
#                     print("Max retries reached. Moving on...")
#         next_annotated_programs_per_problem.append(next_annotated_program)
#         testable_programs_per_problem.append(testable_program)
#     next_programs.append(next_annotated_programs_per_problem)
#     testable_programs.append(testable_programs_per_problem)
    

 10%|▉         | 15/153 [01:14<12:58,  5.64s/it]

Attempt 1 failed: closing parenthesis ']' does not match opening parenthesis '(' on line 40 (eidetivbiylxtsidmjhmywxfvwtjlnnb.py, line 43)
Retrying in 10 seconds...


 10%|█         | 16/153 [01:35<23:39, 10.36s/it]

Attempt 2 failed: closing parenthesis ']' does not match opening parenthesis '(' on line 40 (mbxmmobqqwwjbrinjkawyisysppzejxn.py, line 43)
Max retries reached. Moving on...


100%|██████████| 153/153 [18:56<00:00,  7.43s/it]


In [23]:
# dataset = dataset.add_column("testable_programs", testable_programs)

dataset.push_to_hub("huypn16/LCB-R-F")

Uploading the dataset shards: 100%|██████████| 6/6 [01:46<00:00, 17.76s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/huypn16/LCB-R-F/commit/8a641f39194a222859adf599cf19d0cb80239d5c', commit_message='Upload dataset', commit_description='', oid='8a641f39194a222859adf599cf19d0cb80239d5c', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
generations_all = []
samples = []
responses_all = []

for idx in tqdm(range(len(dataset))):
# for idx in range(1):
    problem = CodeGenerationProblem(dataset[idx])
    generations = []
    for j, buggy_program in enumerate(dataset[idx]["failed_solution"]):
        error_message = dataset[idx]["error_messages"][j]    
        annotated_buggy_program = execute_and_trace(dataset[idx]["testable_programs"][j])
        generations.append(run_next(problem, annotated_buggy_program, error_message))
    generations_all.append(generations)
    samples.append(problem.get_evaluation_sample())
    
results, metadata = evaluate_generations(samples, generations_all, num_process_evaluate=32, timeout=6)
metric = codegen_metrics(samples, generations_all)

 15%|█▌        | 23/153 [00:05<00:17,  7.32it/s]

unable to get function error = (<class 'AttributeError'>, AttributeError("module 'tmp_sol' has no attribute 'smallestString'"), <traceback object at 0x7f69a9befd00>)


 20%|█▉        | 30/153 [00:06<00:12,  9.59it/s]

All tests passed!
All tests passed!

 23%|██▎       | 35/153 [00:06<00:08, 13.53it/s]

...

 99%|█████████▉| 380/383 [00:37<00:03,  1.02s/it]

alarm went off


100%|█████████▉| 382/383 [00:43<00:01,  1.64s/it]

alarm went off


100%|██████████| 383/383 [01:19<00:00,  4.83it/s]


In [19]:
print(dataset[0]["question_content"])

We have a sequence of length N consisting of positive integers: A=(A_1,\ldots,A_N). Any two adjacent terms have different values.
Let us insert some numbers into this sequence by the following procedure.

- If every pair of adjacent terms in A has an absolute difference of 1, terminate the procedure.
- Let A_i, A_{i+1} be the pair of adjacent terms nearest to the beginning of A whose absolute difference is not 1.
- If A_i < A_{i+1}, insert A_i+1,A_i+2,\ldots,A_{i+1}-1 between A_i and A_{i+1}.
- If A_i > A_{i+1}, insert A_i-1,A_i-2,\ldots,A_{i+1}+1 between A_i and A_{i+1}.


- Return to step 1.

Print the sequence when the procedure ends.

Input

The input is given from Standard Input in the following format:
N
A_1 A_2 \ldots A_N

Output

Print the terms in the sequence when the procedure ends, separated by spaces.

Constraints


- 2 \leq N \leq 100
- 1 \leq A_i \leq 100
- A_i \neq A_{i+1}
- All values in the input are integers.

Sample Input 1

4
2 5 1 2

Sample Output 1

2 3 4 5 4 3 2

In [20]:
print(dataset[0]["failed_solution"][0])

import sys

# Read input
N = int(sys.stdin.readline().strip())
A = [int(x) for x in sys.stdin.readline().strip().split()]

# Helper function to insert numbers between adjacent terms
def insert_numbers(arr):
    i = 0
    while i < len(arr) - 1:
        if abs(arr[i] - arr[i+1]) != 1:
            if arr[i] < arr[i+1]:
                for j in range(arr[i]+1, arr[i+1]):
                    arr.insert(i+1, j)
                i += arr[i+1] - arr[i]
            else:
                for j in range(arr[i+1]+1, arr[i]):
                    arr.insert(i+1, j)
                i += arr[i] - arr[i+1]
        else:
            i += 1
    return arr

# Solve the problem
while True:
    A = insert_numbers(A)
    if all(abs(A[i] - A[i+1]) == 1 for i in range(len(A)-1)):
        break

# Print the final sequence
print(" ".join(map(str, A)))


In [14]:
metric[0]

{'pass@1': 0.4570806100217865,
 'detail': {'pass@1': {0: 0.0,
   1: 0.0,
   2: 0.0,
   3: 0.0,
   4: 0.0,
   5: 0.0,
   6: 0.0,
   7: 0.0,
   8: 0.0,
   9: 0.0,
   10: 0.0,
   11: 0.0,
   12: 0.0,
   13: 0.0,
   14: 0.0,
   15: 0.0,
   16: 0.0,
   17: 0.0,
   18: 0.0,
   19: 0.0,
   20: 0.0,
   21: 0.0,
   22: 0.0,
   23: 0.0,
   24: 0.0,
   25: 0.0,
   26: 0.0,
   27: 0.0,
   28: 0.0,
   29: 0.0,
   30: 0.0,
   31: 0.0,
   32: 0.0,
   33: 1.0,
   34: 1.0,
   35: 1.0,
   36: 1.0,
   37: 1.0,
   38: 1.0,
   39: 0.25,
   40: 0.0,
   41: 0.5,
   42: 1.0,
   43: 0.0,
   44: 0.0,
   45: 0.0,
   46: 1.0,
   47: 1.0,
   48: 0.0,
   49: 1.0,
   50: 0.6666666666666666,
   51: 1.0,
   52: 1.0,
   53: 0.5,
   54: 1.0,
   55: 1.0,
   56: 0.0,
   57: 1.0,
   58: 0.33333333333333326,
   59: 0.0,
   60: 1.0,
   61: 1.0,
   62: 1.0,
   63: 1.0,
   64: 0.0,
   65: 1.0,
   66: 1.0,
   67: 0.0,
   68: 0.0,
   69: 1.0,
   70: 0.0,
   71: 1.0,
   72: 1.0,
   73: 0.5,
   74: 1.0,
   75: 0.0,
   76: 1.0,
   